In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import lime
import lime.lime_tabular
import shap
import time
import sys
sys.path.append('../../wrapper')
import xgboost
from xgboost import XGBClassifier
from collections import Counter

import os
import glob
import mglearn
import sklearn
import sklearn.datasets
from numpy import random

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectPercentile

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, cohen_kappa_score, classification_report
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn import pipeline
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids

from string import ascii_letters
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
shap.initjs()

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
y_valid = pd.read_csv("y_valid_Mo2.csv", sep='\t', index_col=0)
X_valid_selected = pd.read_csv("X_valid_selected_Mo2.csv", sep='\t', index_col=0)
X_traintest_selected = pd.read_csv("X_traintest_selected_Mo2.csv", sep='\t', index_col=0)
y_traintest_sampled = pd.read_csv("y_traintest_sampled_Mo2.csv", sep='\t', index_col=0)

In [3]:
params = {'max_depth':20, 'n_estimators':500,
          'objective': 'multi:softmax', 'num_class': 2
}

params['max_depth'] = 9
params['min_child_weight'] = 5
params['n_estimators'] = 100
params['subsample'] = .7
params['colsample_bytree'] = .9

In [4]:
xgb_traintest = xgboost.DMatrix(X_traintest_selected, label=y_traintest_sampled)
xgb_valid = xgboost.DMatrix(X_valid_selected, label=y_valid)

In [5]:
xgb = xgboost.train(params, xgb_traintest, 100)

In [6]:
xgb_pred = xgb.predict(xgb_valid)

In [7]:
Mo2_tp = y_valid.index[(y_valid==1).iloc[:, 0] & (xgb_pred==1)].tolist()
Mo2_fp = y_valid.index[(y_valid==0).iloc[:, 0] & (xgb_pred==1)].tolist()
Mo2_tn = y_valid.index[(y_valid==0).iloc[:, 0] & (xgb_pred==0)].tolist()
Mo2_fn = y_valid.index[(y_valid==1).iloc[:, 0] & (xgb_pred==0)].tolist()

In [8]:
Mo2_fp_samples = random.choice(Mo2_fp, len(Mo2_tp))
Mo2_tn_samples = random.choice(Mo2_tn, len(Mo2_fn))

In [70]:
Mo2_pos = []
Mo2_pos.extend(Mo2_tp)
Mo2_pos.extend(Mo2_fp_samples)
Mo2_neg = []
Mo2_neg.extend(Mo2_fn)
Mo2_neg.extend(Mo2_tn_samples)
Mo2_all = []
Mo2_all.extend(Mo2_pos)
Mo2_all.extend(Mo2_neg)

In [10]:
xgb_valid_Mo2_tp = xgboost.DMatrix(X_valid_selected.iloc[Mo2_tp,:], label=y_valid.loc[Mo2_tp])
xgb_valid_Mo2_fp = xgboost.DMatrix(X_valid_selected.iloc[Mo2_fp_samples,:], label=y_valid.loc[Mo2_fp_samples])
xgb_valid_Mo2_tn = xgboost.DMatrix(X_valid_selected.iloc[Mo2_tn_samples,:], label=y_valid.loc[Mo2_tn_samples])
xgb_valid_Mo2_fn = xgboost.DMatrix(X_valid_selected.iloc[Mo2_fn,:], label=y_valid.loc[Mo2_fn])

In [11]:
shap_values_xgb_Mo2_tp = xgb.predict(xgb_valid_Mo2_tp, pred_contribs=True)
shap_values_xgb_Mo2_fp = xgb.predict(xgb_valid_Mo2_fp, pred_contribs=True)
shap_values_xgb_Mo2_tn = xgb.predict(xgb_valid_Mo2_tn, pred_contribs=True)
shap_values_xgb_Mo2_fn = xgb.predict(xgb_valid_Mo2_fn, pred_contribs=True)

In [71]:
xgb_valid_Mo2_pos = xgboost.DMatrix(X_valid_selected.iloc[Mo2_pos,:], label=y_valid.loc[Mo2_pos])
xgb_valid_Mo2_neg = xgboost.DMatrix(X_valid_selected.iloc[Mo2_neg,:], label=y_valid.loc[Mo2_neg])
xgb_valid_Mo2_all = xgboost.DMatrix(X_valid_selected.iloc[Mo2_all,:], label=y_valid.loc[Mo2_all])

shap_values_xgb_Mo2_pos = xgb.predict(xgb_valid_Mo2_pos, pred_contribs=True)
shap_values_xgb_Mo2_neg = xgb.predict(xgb_valid_Mo2_neg, pred_contribs=True)
shap_values_xgb_Mo2_all = xgb.predict(xgb_valid_Mo2_all, pred_contribs=True)

In [13]:
import warnings
from scipy.stats import gaussian_kde
import sys
# sys.path.append("iml_master/iml_test")
sys.path.insert(0, "iml_master")

import iml_test, iml_test.links
from iml_test import Instance, Model, visualize
from iml_test.explanations import AdditiveExplanation
from iml_test.links import IdentityLink
from iml_test.datatypes import DenseData
import numpy as np

def force_plot(shap_values, features=None, feature_names=None, out_names=None, link="identity",
               plot_cmap="RdBu"):
    """ Visualize the given SHAP values with an additive force layout. """

    assert not type(shap_values) == list, "The shap_values arg looks looks multi output, try shap_values[i]."

    link = iml_test.links.convert_to_link(link)
    if type(shap_values) != np.ndarray:
        return iml_test.visualize(shap_values)

    # convert from a DataFrame or other types
    if str(type(features)) == "<class 'pandas.core.frame.DataFrame'>":
        if feature_names is None:
            feature_names = list(features.columns)
        features = features.as_matrix()
    elif str(type(features)) == "<class 'pandas.core.series.Series'>":
        if feature_names is None:
            feature_names = list(features.index)
        features = features.as_matrix()
    elif str(type(features)) == "list":
        if feature_names is None:
            feature_names = features
        features = None
    elif features is not None and len(features.shape) == 1 and feature_names is None:
        feature_names = features
        features = None

    if len(shap_values.shape) == 1:
        shap_values = np.reshape(shap_values, (1, len(shap_values)))

    if out_names is None:
        out_names = ["output value"]

    if shap_values.shape[0] == 1:
        if feature_names is None:
            feature_names = ["Feature " + str(i) for i in range(shap_values.shape[1] - 1)]
        if features is None:
            features = ["" for _ in range(len(feature_names))]
        if type(features) == np.ndarray:
            features = features.flatten()

        instance = Instance(np.zeros((1, len(feature_names))), features)
        e = AdditiveExplanation(
            shap_values[0, -1],
            np.sum(shap_values[0, :]),
            shap_values[0, :-1],
            None,
            instance,
            link,
            Model(None, out_names),
            DenseData(np.zeros((1, len(feature_names))), list(feature_names))
        )
        return visualize(e, plot_cmap)

    else:
        if shap_values.shape[0] > 3000:
            warnings.warn("shap.force_plot is slow many thousands of rows, try subsampling your data.")

        exps = []
        for i in range(shap_values.shape[0]):
            if feature_names is None:
                feature_names = ["Feature " + str(i) for i in range(shap_values.shape[1] - 1)]
            if features is None:
                display_features = ["" for i in range(len(feature_names))]
            else:
                display_features = features[i, :]

            instance = Instance(np.ones((1, len(feature_names))), display_features)
            e = AdditiveExplanation(
                shap_values[i, -1],
                np.sum(shap_values[i, :]),
                shap_values[i, :-1],
                None,
                instance,
                link,
                Model(None, out_names),
                DenseData(np.ones((1, len(feature_names))), list(feature_names))
            )
            exps.append(e)
        return visualize(exps, plot_cmap=plot_cmap)

In [64]:
from __future__ import unicode_literals
import os
import io
import string
import json
import random
from IPython.core.display import display, HTML
import base64
import numpy as np
import scipy.cluster
from iml_test.explanations import Explanation, AdditiveExplanation
import collections
from iml_test.common import verify_valid_cmap


err_msg = """
<div style='color: #900; text-align: center;'>
  <b>Visualization omitted, Javascript library not loaded!</b><br>
  Have you run `initjs()` in this notebook? If this notebook was from another
  user you must also trust this notebook (File -> Trust notebook). If you are viewing
  this notebook on github the Javascript has been stripped for security.
</div>"""


def initjs():
    bundle_path = os.path.join(os.path.split(__file__)[0], "resources", "bundle.js")
    bundle_data = io.open(bundle_path, encoding="utf-8").read()
    logo_path = os.path.join(os.path.split(__file__)[0], "resources", "logoSmallGray.png")
    logo_data = base64.b64encode(open(logo_path, "rb").read()).decode('utf-8')
    display(HTML(
        "<div align='center'><img src='data:image/png;base64,{logo_data}' /></div>".format(logo_data=logo_data) +
        "<script>{bundle_data}</script>".format(bundle_data=bundle_data)
    ))


def id_generator(size=20, chars=string.ascii_uppercase + string.digits):
    return "i"+''.join(random.choice(chars) for _ in range(size))


def ensure_not_numpy(x):
    if isinstance(x, bytes):
        return x.decode()
    elif isinstance(x, np.str):
        return str(x)
    elif isinstance(x, np.generic):
        return float(np.asscalar(x))
    else:
        return x


def visualize(e, plot_cmap="RdBu"):
    plot_cmap = verify_valid_cmap(plot_cmap)
    if isinstance(e, AdditiveExplanation):
        return AdditiveForceVisualizer(e, plot_cmap=plot_cmap).html()
    elif isinstance(e, Explanation):
        return SimpleListVisualizer(e).html()
    elif isinstance(e, collections.Sequence) and len(e) > 0 and isinstance(e[0], AdditiveExplanation):
        return AdditiveForceArrayVisualizer(e, plot_cmap=plot_cmap).html()
    else:
        assert False, "visualize() can only display Explanation objects (or arrays of them)!"

# try:
#     # register the visualize function with IPython
#     ip = get_ipython()
#     svg_formatter=ip.display_formatter.formatters['text/html']
#     svg_formatter.for_type(Explanation, lambda x: visualize(x).data)
#     old_list_formatter = svg_formatter.for_type(list)
#     def try_list_display(e):
#         if isinstance(e, collections.Sequence) and len(e) > 0 and isinstance(e[0], AdditiveExplanation):
#             return visualize(e).data
#         else:
#             return str(e) if old_list_formatter is None else old_list_formatter(e)
#     svg_formatter.for_type(list, try_list_display)
# except:
#     pass

class SimpleListVisualizer:
    def __init__(self, e):
        assert isinstance(e, Explanation), "SimpleListVisualizer can only visualize Explanation objects!"

        # build the json data
        features = {}
        for i in filter(lambda j: e.effects[j] != 0, range(len(e.data.group_names))):
            features[i] = {
                "effect": e.effects[i],
                "value": e.instance.group_display_values[i]
            }
        self.data = {
            "outNames": e.model.out_names,
            "base_value": e.base_value,
            "link": str(e.link),
            "featureNames": e.data.group_names,
            "features": features,
            "plot_cmap":e.plot_cmap.plot_cmap
        }

    def html(self):
        return HTML("""
<div id='{id}'>{err_msg}</div>
 <script>
   if (window.IML) IML.ReactDom.render(
    IML.React.createElement(IML.SimpleListVisualizer, {data}),
    document.getElementById('{id}')
  );
</script>""".format(err_msg=err_msg, data=json.dumps(self.data), id=id_generator()))


class AdditiveForceVisualizer:
    def __init__(self, e, plot_cmap="RdBu"):
        assert isinstance(e, AdditiveExplanation), \
            "AdditiveForceVisualizer can only visualize AdditiveExplanation objects!"

        # build the json data
        features = {}
        for i in filter(lambda j: e.effects[j] != 0, range(len(e.data.group_names))):
            features[i] = {
                "effect": ensure_not_numpy(e.effects[i]),
                "value": ensure_not_numpy(e.instance.group_display_values[i])
            }
        self.data = {
            "outNames": e.model.out_names,
            "baseValue": ensure_not_numpy(e.base_value),
            "outValue": ensure_not_numpy(e.out_value),
            "link": str(e.link),
            "featureNames": e.data.group_names,
            "features": features,
            "plot_cmap": plot_cmap
        }

    def html(self, label_margin=20):
        self.data["labelMargin"] = label_margin
        return HTML("""
<div id='{id}'>{err_msg}</div>
 <script>
   if (window.IML) IML.ReactDom.render(
    IML.React.createElement(IML.AdditiveForceVisualizer, {data}),
    document.getElementById('{id}')
  );
</script>""".format(err_msg=err_msg, data=json.dumps(self.data), id=id_generator()))


class AdditiveForceArrayVisualizer:
    def __init__(self, arr, plot_cmap="RdBu"):
        assert isinstance(arr[0], AdditiveExplanation), \
            "AdditiveForceArrayVisualizer can only visualize arrays of AdditiveExplanation objects!"

        # order the samples by their position in a hierarchical clustering
        if all([e.model.f == arr[1].model.f for e in arr]):
            #m = np.vstack([e.effects for e in arr])
            D = scipy.spatial.distance.pdist(np.vstack([e.effects for e in arr]), 'sqeuclidean')
            #D = np.vstack([np.sum((m - m[i,:])**2, 1) for i in range(m.shape[0])])
            clustOrder = scipy.cluster.hierarchy.leaves_list(scipy.cluster.hierarchy.complete(D))
        else:
            assert False, "Tried to visualize an array of explanations from different models!"

        # make sure that we put the higher predictions first...just for consistency
        if sum(arr[clustOrder[0]].effects) < sum(arr[clustOrder[-1]].effects):
            np.flipud(clustOrder) # reverse

        # build the json data
        clustOrder = np.argsort(clustOrder) # inverse permutation
        self.data = {
            "outNames": arr[0].model.out_names,
            "baseValue": ensure_not_numpy(arr[0].base_value),
            "link": arr[0].link.__str__(),
            "featureNames": arr[0].data.group_names,
            "explanations": [],
            "plot_cmap": plot_cmap
        }
        for (ind,e) in enumerate(arr):
            self.data["explanations"].append({
                "outValue": ensure_not_numpy(e.out_value),
                "simIndex": ensure_not_numpy(clustOrder[ind])+1,
                "features": {}
            })
            for i in filter(lambda j: e.effects[j] != 0 or e.instance.x[0,j] != 0, range(len(e.data.group_names))):
                self.data["explanations"][-1]["features"][i] = {
                    "effect": ensure_not_numpy(e.effects[i]),
                    "value": ensure_not_numpy(e.instance.group_display_values[i])
                }

    def html(self):
        return HTML("""
<div id='{id}'>{err_msg}</div>
 <script>
   if (window.IML) IML.ReactDom.render(
    IML.React.createElement(IML.AdditiveForceArrayVisualizer, {data}),
    document.getElementById('{id}')
  );
</script>""".format(err_msg=err_msg, data=json.dumps(self.data), id=id_generator()))

In [65]:
force_plot(shap_values_xgb_Mo2_tp[:, 1, :], X_valid_selected.iloc[Mo2_tp,:])

In [22]:
force_plot(shap_values_xgb_Mo2_fp[:, 1, :], X_valid_selected.iloc[Mo2_fp,:])

test


In [23]:
force_plot(shap_values_xgb_Mo2_fn[:, 1, :], X_valid_selected.iloc[Mo2_fn,:])

test


In [24]:
force_plot(shap_values_xgb_Mo2_tn[:, 1, :], X_valid_selected.iloc[Mo2_tn,:])

test


In [17]:
force_plot(shap_values_xgb_Mo2_pos[:, 1, :], X_valid_selected.iloc[Mo2_pos,:])

test


In [68]:
force_plot(shap_values_xgb_Mo2_neg[:, 1, :], X_valid_selected.iloc[Mo2_neg,:])

In [72]:
force_plot(shap_values_xgb_Mo2_all[:, 1, :], X_valid_selected.iloc[Mo2_all,:])

In [79]:
print("TP max=",np.sum(shap_values_xgb_Mo2_tp[:, 1, :], axis=1).max(), "min=", np.sum(shap_values_xgb_Mo2_tp[:, 1, :], axis=1).min())
print("FP max=",np.sum(shap_values_xgb_Mo2_fp[:, 1, :], axis=1).max(), "min=", np.sum(shap_values_xgb_Mo2_fp[:, 1, :], axis=1).min())
print("FN max=",np.sum(shap_values_xgb_Mo2_fn[:, 1, :], axis=1).max(), "min=", np.sum(shap_values_xgb_Mo2_fn[:, 1, :], axis=1).min())
print("TN max=",np.sum(shap_values_xgb_Mo2_tn[:, 1, :], axis=1).max(), "min=", np.sum(shap_values_xgb_Mo2_tn[:, 1, :], axis=1).min())

TP max= 5.73518 min= 0.069911
FP max= 3.6476 min= -0.540122
FN max= 0.508991 min= -4.63483
TN max= -2.28543 min= -6.40206


In [81]:
shap_values_xgb_Mo2_all[:, 1, :].shape

(768, 23)